In [96]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import utils
import catboost
import scoring

In [109]:
np.random.seed(42)
config = {
    'data': {
        'train_full': "./data/train_full.hdf",
        'train_label': "./data/train_label_weight.hdf",
        'test_full': "./data/test_private.hdf",
        'val_size': 0.05
    },
    'model': {
        'params': {
            'iterations': 1000, ##25000 for track 1
            'thread_count': 16,
            'use_best_model': True,
            'early_stopping_rounds': 100,
            'eval_metric': 'Logloss'
        }
    }
}

In [110]:
def load_data(train_full=None, train_label=None, test_full=None):
    train_full = pd.read_hdf(train_full)
    train_label_weight = pd.read_hdf(train_label)
    train_full['weight'] = train_label_weight.weight
    train_full['label']  = train_label_weight.label
    test_full = pd.read_hdf(test_full)
    
    return train_full, test_full

In [111]:
def to_catboost_pool(df, feature_columns, label_column=None):
    pool = catboost.Pool(
        data=df.loc[:, feature_columns],
        label=df[label_column] if label_column is not None else None
    )
    return pool

In [112]:
def load_data_pool(train_full=None, train_label=None, test_full=None, val_size=None):
    
    train_df, test_df = load_data(train_full, train_label, test_full)
    train_df, val_df = train_test_split(train_df, test_size=val_size, shuffle=True)
    
    FEATURES = [col for col in list(train_df.columns) if col not in utils.TRAIN_COLUMNS]
    
    train_pool = to_catboost_pool(train_df, FEATURES, 'label')
    val_pool = to_catboost_pool(val_df, FEATURES, 'label')
    test_pool = to_catboost_pool(test_df, FEATURES)
    test_index = test_df.index
    val_weight = val_df.weight.values
    val_labels = val_df.label.values
    
    train_pool.set_weight(np.abs(train_df.weight.values))
    
    return train_pool, val_pool, val_weight, val_labels, test_pool, test_index

In [113]:
train, val, val_weight, val_labels, test, test_index = load_data_pool(**config['data'])

In [114]:
model = catboost.CatBoostClassifier(**config['model']['params'])

In [ ]:
model.fit(X=train, plot=True,
          eval_set=val, verbose=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.048292
0:	learn: 0.6794844	test: 0.6917227	best: 0.6917227 (0)	total: 714ms	remaining: 4h 57m 35s
1:	learn: 0.6665446	test: 0.6892605	best: 0.6892605 (1)	total: 1.4s	remaining: 4h 50m 43s
2:	learn: 0.6556341	test: 0.6884377	best: 0.6884377 (2)	total: 2.01s	remaining: 4h 38m 46s
3:	learn: 0.6461892	test: 0.6878274	best: 0.6878274 (3)	total: 2.62s	remaining: 4h 32m 24s
4:	learn: 0.6377248	test: 0.6878067	best: 0.6878067 (4)	total: 3.24s	remaining: 4h 29m 50s
5:	learn: 0.6301863	test: 0.6876267	best: 0.6876267 (5)	total: 3.79s	remaining: 4h 23m 26s
6:	learn: 0.6228616	test: 0.6871414	best: 0.6871414 (6)	total: 4.38s	remaining: 4h 20m 43s
7:	learn: 0.6153913	test: 0.6866684	best: 0.6866684 (7)	total: 5.04s	remaining: 4h 22m 21s
8:	learn: 0.6090329	test: 0.6859712	best: 0.6859712 (8)	total: 5.61s	remaining: 4h 19m 32s
9:	learn: 0.6037415	test: 0.6864032	best: 0.6859712 (8)	total: 6.26s	remaining: 4h 20m 41s
10:	learn: 0.5978697	test: 0.6854436	best: 0.6854436 (10)	tot

89:	learn: 0.4976058	test: 0.6513282	best: 0.6513282 (89)	total: 55.8s	remaining: 4h 17m 31s
90:	learn: 0.4971736	test: 0.6508750	best: 0.6508750 (90)	total: 56.4s	remaining: 4h 17m 12s
91:	learn: 0.4969870	test: 0.6505626	best: 0.6505626 (91)	total: 56.9s	remaining: 4h 16m 44s
92:	learn: 0.4966124	test: 0.6501452	best: 0.6501452 (92)	total: 57.5s	remaining: 4h 16m 52s
93:	learn: 0.4962717	test: 0.6497222	best: 0.6497222 (93)	total: 58.1s	remaining: 4h 16m 41s
94:	learn: 0.4959983	test: 0.6493999	best: 0.6493999 (94)	total: 58.8s	remaining: 4h 16m 51s
95:	learn: 0.4957092	test: 0.6490274	best: 0.6490274 (95)	total: 59.4s	remaining: 4h 16m 53s
96:	learn: 0.4954637	test: 0.6486830	best: 0.6486830 (96)	total: 1m	remaining: 4h 16m 56s
97:	learn: 0.4952383	test: 0.6484069	best: 0.6484069 (97)	total: 1m	remaining: 4h 16m 40s
98:	learn: 0.4950334	test: 0.6481283	best: 0.6481283 (98)	total: 1m 1s	remaining: 4h 16m 24s
99:	learn: 0.4948351	test: 0.6478205	best: 0.6478205 (99)	total: 1m 1s	remai

175:	learn: 0.4793778	test: 0.6278613	best: 0.6278613 (175)	total: 1m 47s	remaining: 4h 12m 22s
176:	learn: 0.4792739	test: 0.6277189	best: 0.6277189 (176)	total: 1m 47s	remaining: 4h 12m 16s
177:	learn: 0.4792065	test: 0.6275889	best: 0.6275889 (177)	total: 1m 48s	remaining: 4h 13m 17s
178:	learn: 0.4791341	test: 0.6274962	best: 0.6274962 (178)	total: 1m 49s	remaining: 4h 13m 2s
179:	learn: 0.4790281	test: 0.6273969	best: 0.6273969 (179)	total: 1m 50s	remaining: 4h 12m 50s
180:	learn: 0.4789466	test: 0.6272787	best: 0.6272787 (180)	total: 1m 50s	remaining: 4h 12m 42s
181:	learn: 0.4788312	test: 0.6271922	best: 0.6271922 (181)	total: 1m 51s	remaining: 4h 12m 41s
182:	learn: 0.4787107	test: 0.6270436	best: 0.6270436 (182)	total: 1m 51s	remaining: 4h 12m 31s
183:	learn: 0.4785897	test: 0.6269296	best: 0.6269296 (183)	total: 1m 52s	remaining: 4h 12m 29s
184:	learn: 0.4785223	test: 0.6268261	best: 0.6268261 (184)	total: 1m 52s	remaining: 4h 12m 20s
185:	learn: 0.4784091	test: 0.6266793	bes

262:	learn: 0.4722489	test: 0.6186900	best: 0.6186900 (262)	total: 2m 38s	remaining: 4h 9m 2s
263:	learn: 0.4721333	test: 0.6185643	best: 0.6185643 (263)	total: 2m 39s	remaining: 4h 9m 4s
264:	learn: 0.4720739	test: 0.6184865	best: 0.6184865 (264)	total: 2m 40s	remaining: 4h 9m
265:	learn: 0.4719671	test: 0.6183853	best: 0.6183853 (265)	total: 2m 40s	remaining: 4h 8m 52s
266:	learn: 0.4719044	test: 0.6182790	best: 0.6182790 (266)	total: 2m 41s	remaining: 4h 8m 52s
267:	learn: 0.4718399	test: 0.6181764	best: 0.6181764 (267)	total: 2m 41s	remaining: 4h 8m 45s
268:	learn: 0.4717699	test: 0.6180687	best: 0.6180687 (268)	total: 2m 42s	remaining: 4h 8m 45s
269:	learn: 0.4717355	test: 0.6180347	best: 0.6180347 (269)	total: 2m 42s	remaining: 4h 8m 40s
270:	learn: 0.4716801	test: 0.6179618	best: 0.6179618 (270)	total: 2m 43s	remaining: 4h 8m 42s
271:	learn: 0.4716177	test: 0.6179046	best: 0.6179046 (271)	total: 2m 44s	remaining: 4h 8m 36s
272:	learn: 0.4715732	test: 0.6178534	best: 0.6178534 (2

...

21681:	learn: 0.2853480	test: 0.4013231	best: 0.4013231 (21681)	total: 4h 30m 31s	remaining: 41m 23s
21682:	learn: 0.2853449	test: 0.4013197	best: 0.4013197 (21682)	total: 4h 30m 32s	remaining: 41m 23s
21683:	learn: 0.2853417	test: 0.4013180	best: 0.4013180 (21683)	total: 4h 30m 33s	remaining: 41m 22s
21684:	learn: 0.2853388	test: 0.4013161	best: 0.4013161 (21684)	total: 4h 30m 33s	remaining: 41m 21s
21685:	learn: 0.2853345	test: 0.4013118	best: 0.4013118 (21685)	total: 4h 30m 34s	remaining: 41m 20s
21686:	learn: 0.2853314	test: 0.4013073	best: 0.4013073 (21686)	total: 4h 30m 35s	remaining: 41m 20s
21687:	learn: 0.2853274	test: 0.4013014	best: 0.4013014 (21687)	total: 4h 30m 36s	remaining: 41m 19s
21688:	learn: 0.2853246	test: 0.4012981	best: 0.4012981 (21688)	total: 4h 30m 37s	remaining: 41m 18s
21689:	learn: 0.2853196	test: 0.4012927	best: 0.4012927 (21689)	total: 4h 30m 37s	remaining: 41m 17s
21690:	learn: 0.2853162	test: 0.4012887	best: 0.4012887 (21690)	total: 4h 30m 38s	remaining

21763:	learn: 0.2850441	test: 0.4009807	best: 0.4009807 (21763)	total: 4h 31m 37s	remaining: 40m 23s
21764:	learn: 0.2850402	test: 0.4009770	best: 0.4009770 (21764)	total: 4h 31m 38s	remaining: 40m 22s
21765:	learn: 0.2850361	test: 0.4009728	best: 0.4009728 (21765)	total: 4h 31m 39s	remaining: 40m 21s
21766:	learn: 0.2850310	test: 0.4009677	best: 0.4009677 (21766)	total: 4h 31m 39s	remaining: 40m 20s
21767:	learn: 0.2850269	test: 0.4009623	best: 0.4009623 (21767)	total: 4h 31m 40s	remaining: 40m 20s
21768:	learn: 0.2850225	test: 0.4009560	best: 0.4009560 (21768)	total: 4h 31m 41s	remaining: 40m 19s
21769:	learn: 0.2850180	test: 0.4009514	best: 0.4009514 (21769)	total: 4h 31m 42s	remaining: 40m 18s
21770:	learn: 0.2850149	test: 0.4009483	best: 0.4009483 (21770)	total: 4h 31m 42s	remaining: 40m 17s
21771:	learn: 0.2850109	test: 0.4009438	best: 0.4009438 (21771)	total: 4h 31m 43s	remaining: 40m 17s
21772:	learn: 0.2850067	test: 0.4009392	best: 0.4009392 (21772)	total: 4h 31m 44s	remaining

21845:	learn: 0.2847510	test: 0.4006612	best: 0.4006612 (21845)	total: 4h 32m 44s	remaining: 39m 22s
21846:	learn: 0.2847479	test: 0.4006568	best: 0.4006568 (21846)	total: 4h 32m 45s	remaining: 39m 21s
21847:	learn: 0.2847444	test: 0.4006519	best: 0.4006519 (21847)	total: 4h 32m 46s	remaining: 39m 21s
21848:	learn: 0.2847411	test: 0.4006482	best: 0.4006482 (21848)	total: 4h 32m 47s	remaining: 39m 20s
21849:	learn: 0.2847373	test: 0.4006433	best: 0.4006433 (21849)	total: 4h 32m 48s	remaining: 39m 19s
21850:	learn: 0.2847333	test: 0.4006399	best: 0.4006399 (21850)	total: 4h 32m 48s	remaining: 39m 18s
21851:	learn: 0.2847299	test: 0.4006384	best: 0.4006384 (21851)	total: 4h 32m 49s	remaining: 39m 18s
21852:	learn: 0.2847263	test: 0.4006334	best: 0.4006334 (21852)	total: 4h 32m 50s	remaining: 39m 17s
21853:	learn: 0.2847234	test: 0.4006305	best: 0.4006305 (21853)	total: 4h 32m 51s	remaining: 39m 16s
21854:	learn: 0.2847188	test: 0.4006257	best: 0.4006257 (21854)	total: 4h 32m 52s	remaining

21927:	learn: 0.2844625	test: 0.4003550	best: 0.4003550 (21927)	total: 4h 33m 51s	remaining: 38m 22s
21928:	learn: 0.2844587	test: 0.4003507	best: 0.4003507 (21928)	total: 4h 33m 52s	remaining: 38m 21s
21929:	learn: 0.2844556	test: 0.4003453	best: 0.4003453 (21929)	total: 4h 33m 53s	remaining: 38m 20s
21930:	learn: 0.2844514	test: 0.4003395	best: 0.4003395 (21930)	total: 4h 33m 54s	remaining: 38m 19s
21931:	learn: 0.2844502	test: 0.4003394	best: 0.4003394 (21931)	total: 4h 33m 54s	remaining: 38m 19s
21932:	learn: 0.2844473	test: 0.4003365	best: 0.4003365 (21932)	total: 4h 33m 55s	remaining: 38m 18s
21933:	learn: 0.2844443	test: 0.4003324	best: 0.4003324 (21933)	total: 4h 33m 56s	remaining: 38m 17s
21934:	learn: 0.2844405	test: 0.4003251	best: 0.4003251 (21934)	total: 4h 33m 57s	remaining: 38m 16s
21935:	learn: 0.2844365	test: 0.4003213	best: 0.4003213 (21935)	total: 4h 33m 58s	remaining: 38m 16s
21936:	learn: 0.2844328	test: 0.4003163	best: 0.4003163 (21936)	total: 4h 33m 58s	remaining

22009:	learn: 0.2841674	test: 0.4000292	best: 0.4000292 (22009)	total: 4h 34m 59s	remaining: 37m 21s
22010:	learn: 0.2841633	test: 0.4000268	best: 0.4000268 (22010)	total: 4h 35m	remaining: 37m 20s
22011:	learn: 0.2841609	test: 0.4000233	best: 0.4000233 (22011)	total: 4h 35m 1s	remaining: 37m 19s
22012:	learn: 0.2841567	test: 0.4000190	best: 0.4000190 (22012)	total: 4h 35m 2s	remaining: 37m 19s
22013:	learn: 0.2841548	test: 0.4000180	best: 0.4000180 (22013)	total: 4h 35m 3s	remaining: 37m 18s
22014:	learn: 0.2841509	test: 0.4000152	best: 0.4000152 (22014)	total: 4h 35m 3s	remaining: 37m 17s
22015:	learn: 0.2841486	test: 0.4000133	best: 0.4000133 (22015)	total: 4h 35m 4s	remaining: 37m 17s
22016:	learn: 0.2841450	test: 0.4000088	best: 0.4000088 (22016)	total: 4h 35m 5s	remaining: 37m 16s
22017:	learn: 0.2841418	test: 0.4000045	best: 0.4000045 (22017)	total: 4h 35m 6s	remaining: 37m 15s
22018:	learn: 0.2841376	test: 0.4000020	best: 0.4000020 (22018)	total: 4h 35m 7s	remaining: 37m 14s
22

In [118]:
validation_predictions = model.predict_proba(data=val)[:, 1]

scoring.rejection90(val_labels, validation_predictions, sample_weight=val_weight)

0.7857959475772193

## Creating submission for track 2

In [25]:
model.save_model("./track_2_baseline/track_2_model.cbm")

In [26]:
!cd track_2_baseline && zip -1r track_2_updated_dataset.zip ./*

  adding: Makefile (deflated 53%)
  adding: baseline.cpp (deflated 47%)
  adding: compute_features.cpp (deflated 46%)
  adding: parser.cpp (deflated 68%)
  adding: parser.h (deflated 54%)
  adding: reference_header.h (deflated 67%)
  adding: ripped_evaluator/ (stored 0%)
  adding: ripped_evaluator/evaluator.cpp (deflated 68%)
  adding: ripped_evaluator/model_generated.h (deflated 83%)
  adding: ripped_evaluator/include/ (stored 0%)
  adding: ripped_evaluator/include/flatbuffers/ (stored 0%)
  adding: ripped_evaluator/include/flatbuffers/flatbuffers.h (deflated 71%)
  adding: ripped_evaluator/include/flatbuffers/flexbuffers.h (deflated 70%)
  adding: ripped_evaluator/include/flatbuffers/flatc.h (deflated 60%)
  adding: ripped_evaluator/include/flatbuffers/code_generators.h (deflated 59%)
  adding: ripped_evaluator/include/flatbuffers/minireflect.h (deflated 70%)
  adding: ripped_evaluator/include/flatbuffers/reflection_generated.h (deflated 85%)
  adding: ripped_evaluator/include/flatbu

In [27]:
!mv ./track_2_baseline/track_2_updated_dataset.zip ./submissions

## Creating submission for track 1(NOTE: for track 1 i used 25000 iterations instead of 1000 for track 2)

In [119]:
predictions = model.predict_proba(test)[:, 1]

pd.DataFrame(data={"prediction": predictions}, index=test_index).to_csv(
    "./submissions/submission_catboost_25ktrees_private.csv", index_label="id")